Small Image Classification using Artificial Neural Networks & GPU Benchmarking

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

GPU Testing

In [ ]:
tf.config.list_physical_devices()

In [ ]:
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")
print(f"TensorFlow: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs available: {len(gpus)}")

Loading Image Classification Dataset and Preprocessing

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train.shape, y_train.shape

In [ ]:
# Image plotting function
def plot_images(index, data):
    plt.figure(figsize=(10,1))
    plt.imshow(data[index])
    
plot_images(7, x_train)

In [ ]:
# classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck' ]

In [ ]:
classes[y_train[8][0]] # image at index 8 is of ship
plot_images(8, x_train) # verfying by plotting corresponding image of x

In [ ]:
# scaling the x data
x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0

# One Hot encoding of y data
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=len(classes), dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=len(classes), dtype='float32'
)

Model Building and Training

In [ ]:
# Model Function
def build_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32, 32, 3)), # 3072
        keras.layers.Dense(2100, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy', # used when multiple classes are present
        metrics=['accuracy']
    )
    return model


In [ ]:
# Model Building and Training
model = build_model()
model.fit(x_train_scaled, y_train_categorical, epochs=50)

In [ ]:
model.evaluate(x_test_scaled, y_test_categorical)

In [ ]:
classes[np.argmax(model.predict(np.expand_dims(x_test_scaled[2], axis=0)))], classes[y_test[2][0]]
# plot_images(2, x_test)
y_pred = np.argmax(model.predict(x_test_scaled), axis=1)

Measuring Training Time on GPU vs CPU

In [ ]:
%%timeit -n1 -r1
with tf.device('/CPU:0'):
    cpu_model = build_model()
    cpu_model.fit(x_train_scaled, y_train_categorical, epochs=31)

In [ ]:
%%timeit -n1 -r1
with tf.device('/GPU:0'):
    cpu_model = build_model()
    cpu_model.fit(x_train_scaled, y_train_categorical, epochs=31)

In [ ]:
# Classification Report and Confusion Matrix
def print_report(y_test, y_pred):
    from sklearn.metrics import classification_report, confusion_matrix
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    
print_report(y_test, y_pred)

# Poor Performance of ANN on Image Data

Timing Results:
GPU - 4m 39.8s / CPU - 44m 12.6s